In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# import os; os.chdir('C:\\work\\dev\\python\\progs\\scraper\\ufc\\')
import os; os.chdir('/home/jovyan/work/ufc/')

In [17]:
import pandas as pd

In [18]:
import pandas as pd
import joblib
from ruamel.yaml import YAML
import re
import numpy as np

conf = YAML().load(open('params.yaml'))

# Инференс

In [19]:
from src.inference import get_features_event


ev_nm = 'UFC Fight Night: Уиттакер - Де Риддер'

feat_pred_df = get_features_event(ev_nm=ev_nm, 
                                    fights_fn=conf['inference']['fights_fn'], 
                                    fighters_fn=conf['inference']['fighters_fn'], 
                                    coef_fn=conf['inference']['coef_fn'], 
                                    vocab_fn=conf['inference']['vocab_fn'], 
                                    ufc_rankings_fn=conf['inference']['rankings_fn'], 
                                    rankings_big_fn=conf['inference']['rankings_big_fn'], conf=conf)



/home/jovyan/work/ufc/src/prep_funcs.py:94: DtypeWarning: Columns (205,206,207,208) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fights_in_fn)
/home/jovyan/work/ufc/src/prep_funcs.py:167: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_all['ctrl_stat'].fillna(0, inplace=True)
/home/jovyan/work/ufc/src/prep_funcs.py:173: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pan

## делаем ставку (place_bet)

In [20]:
model = joblib.load(conf['train_eval']['model_fn'])
feats = joblib.load(conf['feat_shortlist_sel']['sign_feat_short_fn'])

feat_pred_df['score1'] = model.predict_proba(feat_pred_df[feats])[:,1]

In [21]:
feat_pred_df = feat_pred_df.assign(fighters = feat_pred_df.apply(lambda x: ' '.join(sorted([x['fighter'], x['opponent']])) , axis=1))
# метрики сохраняем
feat_pred_df['score2'] = 1 - feat_pred_df['score1']

In [22]:
# добавляем такой же df по fighters event, но фильтруя строки с одними и теми же боями
# тогда score2 как раз и будет обратный скор
feat_pred_df = feat_pred_df.drop(columns='score2').merge(feat_pred_df[['fighters', 'event', 'fighter', 'score2']].copy()\
                                .rename(columns={'fighter':'fi'}), on=['fighters', 'event'])\
                                .loc[lambda x: x['fighter']!=x['fi']]\
    .assign(score1=lambda x: (x['score1']+x['score2'])/2)\
    .drop(columns=['fi', 'score2'])

In [23]:
feat_pred_df = feat_pred_df.drop_duplicates(subset='fighters', keep='first').drop(columns='fighters')


In [24]:
from src.inference import place_bet

bet_df = place_bet(placebet_df=feat_pred_df, strategy_selection = pd.Series([True]*len(feat_pred_df), index=feat_pred_df.index), alpha=0.9)


# Пример инференса

In [6]:
import pandas as pd

from ruamel.yaml import YAML
import re
import numpy as np

conf = YAML().load(open('params.yaml'))

# Схема

In [7]:
%load_ext nb_js_diagrammers

In [8]:
%%mermaid_magic -h 300

stateDiagram
    direction TB
    state pred_df {
      direction RL
       vocab_df --> coefs_df
    }
    rating_dfs
    fights_df
    note right of fights_df
        calc agg stats for last record
        end note
    note left of rating_dfs
            define left right fighter
        end note
    rating_dfs --> pred_df
    fights_df-->fighters_df

## Загружаем коэффициенты

In [9]:
query_name = 'UFC Fight Night: Уиттакер - Де Риддер'

coef_df = pd.read_excel(conf['inference']['coef_fn'])
coef_df = coef_df.assign(event_day=lambda x:pd.to_datetime(x['date'], format='%d.%m.%Y'))
if not query_name:
    pred_df = coef_df[coef_df['event_day']==coef_df['event_day'].max()]
else:
    pred_df = coef_df[coef_df['name']==query_name]

In [10]:
# Загружаем словарь бойцов на английском
vocab_df = pd.read_csv(conf['inference']['vocab_fn'])

pred_df = pred_df.merge(vocab_df[['name_rus','fighters_name', 'fights_name', 'ranks_name']].rename(columns={**{'name_rus':'fighter1'}, **{it:f'{it}1' for it in ['fighters_name', 'fights_name', 'ranks_name']}})
                        , on='fighter1', how='left')\
        .merge(vocab_df[['name_rus','fighters_name', 'fights_name', 'ranks_name']].rename(columns={**{'name_rus':'fighter2'}, **{it:f'{it}2' for it in ['fighters_name', 'fights_name', 'ranks_name']}}), on='fighter2', how='left')

pred_df = pred_df.assign(event_day=pd.to_datetime(pred_df['event_day'], format='%Y-%m-%d').dt.to_period(freq='D'))


## рейтинги

### маленький

In [11]:
# подгружаем ранги с датой <= событию и максимальной из оставшихся
event_date = pred_df['event_day'].iloc[0]

rank_df = pd.read_csv(conf['inference']['rankings_fn']).loc[lambda x: ~x['division_name'].isin(['Вне зависимости от категорий Top Rank', 'Женский, вне весовых категорий Top Rank'])]
rank_df['date'] = pd.to_datetime(rank_df['date'], format='%Y-%m-%d').dt.to_period(freq='D')
rank_df = rank_df.groupby(['name', 'date'], as_index=False)['rank'].min()

rank_df = rank_df[rank_df['date']<event_date - 1]
rank_df = rank_df[rank_df['date'] == rank_df['date'].max()]

In [12]:
assert rank_df['name'].duplicated().sum()==0

In [13]:
pred_df = pred_df.merge(rank_df[['name', 'rank']].rename(columns={'name':'fighter1', 'rank':'rank1'}), on='fighter1', how='left')\
       .merge(rank_df[['name', 'rank']].rename(columns={'name':'fighter2', 'rank':'rank2'}), on='fighter2', how='left')
        

### большой

In [14]:
rank_big_df = pd.read_csv('data/inference/rank_big_df.csv', dtype='str')
rank_big_df['date'] = pd.to_datetime(rank_big_df['date'], format='%Y-%m-%d').dt.to_period(freq='D')
rank_big_df['name'] = rank_big_df['name'].str.replace(r'"\s*[^"]*"\s*', '', regex=True)
rank_big_df['rank'] = rank_big_df['rank'].astype('float')

rank_big_df = rank_big_df[rank_big_df['date']<event_date-1]
rank_big_df = rank_big_df[rank_big_df['date']==rank_big_df['date'].max()]

# считаем итоговый рейтинг по лучшему значению
rank_big_df = rank_big_df.groupby(['name', 'date'], as_index=False)['rank'].min()

In [15]:
assert rank_big_df['name'].duplicated().sum()==0

In [16]:
pred_df = pred_df.merge(rank_big_df[['name', 'rank']].rename(columns={'name':'ranks_name1', 'rank':'rank_big1'}), on='ranks_name1', how='left')\
       .merge(rank_big_df[['name', 'rank']].rename(columns={'name':'ranks_name2', 'rank':'rank_big2'}), on='ranks_name2', how='left')

pred_df['left_corner'] = np.nan
sel_ranks = pred_df['rank1'].notna()&pred_df['rank2'].notna()
sel_big_ranks = pred_df['rank_big1'].notna()&pred_df['rank_big2'].notna()
pred_df['left_corner_little'] = np.where((pred_df['rank1']<pred_df['rank2'])&sel_ranks, 1, np.where(sel_ranks, 0, np.nan))
pred_df['left_corner_big'] = np.where((pred_df['rank_big1']<pred_df['rank_big2'])&(sel_big_ranks), 1, np.where(sel_big_ranks, 0, np.nan))
pred_df['left_corner'] = pred_df['left_corner_little'].mask(pred_df['left_corner_little'].isna(), pred_df['left_corner_big'])

## Загружаем поединки и признаки добавляем

In [17]:
from src.prep_funcs import prep_strikes_fn

fighters_in = pd.concat([pred_df[['fights_name1']].rename(columns={'fights_name1':'name'}), pred_df[['fights_name2']].rename(columns={'fights_name2':'name'})])\
                .dropna()\
                ['name'].tolist()

fights_df = prep_strikes_fn(fights_in_fn=conf['inference']['fights_fn'], fighters=fighters_in)

fights_df = fights_df[fights_df['event_date']<event_date-1]

C:\work\dev\python\progs\scraper\ufc\src\prep_funcs.py:94: DtypeWarning: Columns (205,206,207,208) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fights_in_fn)


In [18]:
fights_df = fights_df.sort_values(by=['Fighter', 'event_date'])
stat_cols = [it for it in fights_df.columns if re.search('_stat', it)]
aggs = eval(conf['stat_feat_gen']['aggs'])
cust_aggs = eval(conf['stat_feat_gen']['cust_aggs'])

In [19]:
fights_add_df = pd.concat([pred_df[['fights_name1', 'fights_name2', 'event_day', 'name']].rename(columns={'fights_name1':'Fighter', 'fights_name2':'Opponent', 'event_day':
                                                                                 'event_date', 'name':'Event'}),
           pred_df[['fights_name2', 'fights_name1', 'event_day', 'name']].rename(columns={'fights_name2':'Fighter', 'fights_name1':'Opponent', 'event_day':
                                                                                 'event_date', 'name':'Event'})], ignore_index=True)

fights_df = pd.concat([fights_df, fights_add_df], ignore_index=True)


In [20]:
from src.stat_funcs import get_stat_feat, last_el


fights_stat_df = fights_df.groupby('Fighter').apply(lambda x: get_stat_feat(x, aggs = aggs, cust_aggs=cust_aggs, cols = stat_cols,
                                                min_per_num=conf['stat_feat_gen']['min_fights_num'],
                                                rol_window_size=conf['stat_feat_gen']['rol_window_size']).iloc[[-1]])


fights_stat0_df = fights_df[['Fighter', 'event_date']].copy()

fights_stat0_df['days_nofight_stat'] = fights_df.groupby('Fighter', as_index=False)['event_date'].diff().map(lambda x: x.n, na_action='ignore')
fights_stat0_df['days_nofight_stat'] = fights_stat0_df['days_nofight_stat'].replace({pd.NaT: np.nan}).astype('float32')

fights_stat0_df = fights_stat0_df.groupby('Fighter').apply(lambda x: get_stat_feat(x, aggs = aggs, cust_aggs=cust_aggs+[('last', last_el)], cols = ['days_nofight_stat'],
                                                min_per_num=conf['stat_feat_gen']['min_fights_num'],
                                                rol_window_size=conf['stat_feat_gen']['rol_window_size'], n_shift=0).iloc[[-1]])

fights_stat0_df = fights_stat0_df.assign(days_nofight_stat_custom=lambda x: x['days_nofight_stat_rol_last'])\
                                .drop(columns=['days_nofight_stat_rol_last', 'days_nofight_stat_exp_last'])

fights_stat_df.index = fights_stat_df.index.map(lambda x: x[0])
fights_stat0_df.index = fights_stat0_df.index.map(lambda x: x[0])

fights_stat = fights_stat0_df.merge(fights_stat_df, left_index=True, right_index=True).reset_index(names='full_name')

---

In [57]:
fights_stat[fights_stat.full_name=='Reinier de Ridder']

,full_name,days_nofight_stat_rol_sum,days_nofight_stat_exp_sum,days_nofight_stat_rol_max_min_d,days_nofight_stat_exp_max_min_d,days_nofight_stat_custom,kd_stat_rol_sum,kd_dam_stat_rol_sum,sub_att_stat_rol_sum,sub_att_dam_stat_rol_sum,...,ctrl_stat_exp_max_min_d,ctrl_dam_stat_exp_max_min_d,win_stat_exp_max_min_d,lose_stat_exp_max_min_d,wrest_w_stat_exp_max_min_d,wrest_l_stat_exp_max_min_d,KO_w_stat_exp_max_min_d,KO_l_stat_exp_max_min_d,dec_w_stat_exp_max_min_d,dec_l_stat_exp_max_min_d
156,Reinier de Ridder,259.0,259.0,35.0,35.0,84.0,0.230506,0.0,0.369587,0.0,...,0.674689,0.348668,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [59]:
fights_df[fights_df.Fighter=='Reinier de Ridder']

,Event,Fighter,Opponent,Result,Method:,Round:,Time:,Time format:,Referee:,Details:,...,time,win_stat,lose_stat,method_type,wrest_w_stat,wrest_l_stat,KO_w_stat,KO_l_stat,dec_w_stat,dec_l_stat
285,UFC Fight Night: Magny vs. Prates,Reinier de Ridder,Gerald Meerschaert,W,Submission,3.0,1:44,3 Rnd (5-5-5),Mike Beltran,Arm Triangle From Mount,...,11.733333,1.0,0.0,wrestling,1.0,0.0,0.0,0.0,0.0,0.0
286,UFC 311: Makhachev vs. Moicano,Reinier de Ridder,Kevin Holland,W,Submission,1.0,3:31,3 Rnd (5-5-5),Frank Trigg,Rear Naked Choke,...,3.516667,1.0,0.0,wrestling,1.0,0.0,0.0,0.0,0.0,0.0
287,UFC Fight Night: Sandhagen vs. Figueiredo,Reinier de Ridder,Bo Nickal,W,KO/TKO,2.0,1:53,3 Rnd (5-5-5),Mike Beltran,Knee to Body At Distance,...,6.883333,1.0,0.0,KO/TKO,0.0,0.0,1.0,0.0,0.0,0.0
391,UFC Fight Night: Уиттакер - Де Риддер,Reinier de Ridder,Robert Whittaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

## Загружаем перечень бойцов на английском

In [48]:
from src.prep_funcs import prep_fighters_fn

fighters_in = pd.concat([pred_df[['fighters_name1']].rename(columns={'fighters_name1':'name'}), pred_df[['fighters_name2']].rename(columns={'fighters_name2':'name'})])\
                .dropna()\
                ['name'].tolist()
fighters_df = prep_fighters_fn(conf['inference']['fighters_fn'], fighters=fighters_in)


In [49]:
fighters_df['dob_diff_stat_custom'] = fighters_df['dob'].map(lambda x: x.start_time.timestamp()/(30*24*3600) if not pd.isnull(x) else np.nan)

In [ ]:
stat_df = fighters_df.merge(fights_stat, on='full_name', how='left')

# переименовать колонки
stat_df = stat_df.rename(columns={'height_stat':'height_stat_custom', 'reach_stat':'reach_stat_custom'})

feat_cols = [it for it in stat_df.columns if '_stat_rol' in it or '_stat_exp' in it or '_stat_custom' in it]

[it for it in stat_df.columns if not it in feat_cols]

['full_name', 'stance', 'dob']

## Признаки боев

In [45]:
df1 = pred_df.merge(stat_df, how='left', left_on='fighters_name1', right_on='full_name')\
        .rename(columns={'fighters_name1':'fighter', 'fighters_name2':'opponent'})[['fighter', 'opponent', 'event_day']+feat_cols]\
        .set_index(['fighter', 'opponent', 'event_day'])

df2 = pred_df.merge(stat_df, how='left', left_on='fighters_name2', right_on='full_name')\
        .rename(columns={'fighters_name1':'fighter', 'fighters_name2':'opponent'})[['fighter', 'opponent', 'event_day']+feat_cols]\
        .set_index(['fighter', 'opponent', 'event_day'])


In [46]:
pred_fightes_df = (df1 - df2).dropna()


pred_fightes_df.columns=[f"{it.replace('_stat', '')}_feat" for it in pred_fightes_df.columns]

pred_fightes_df = stat_df.rename(columns={'full_name':'fighter'})[['fighter', 'stance',	'dob']].merge(pred_fightes_df.reset_index(), on='fighter', how='right')
pred_fightes_df = pred_fightes_df.assign(fighter_dob_custom_feat = lambda x: (x['event_day'] - x['dob']).map(lambda y: y.n/30))\
                .rename(columns={'stance':'fighter_stance_custom_feat'})\
                .drop(columns=['dob'])


pred_fightes_df = stat_df.rename(columns={'full_name':'opponent'})[['opponent', 'stance']].merge(pred_fightes_df, on='opponent', how='right')\
        .rename(columns={'stance':'opponent_stance_custom_feat'})

pred_fightes_df = pred_fightes_df.merge(pred_df[['fighters_name1', 'left_corner']].rename(columns={'left_corner':'left_corner_custom_feat', 'fighters_name1':'fighter'}),
                      how='left', on='fighter')

feats = [it for it in pred_fightes_df.columns if '_feat' in it]

pred_fightes_df = pred_fightes_df[['fighter', 'opponent', 'event_day']+feats]

In [47]:
pred_fightes_df

,fighter,opponent,event_day,opponent_stance_custom_feat,fighter_stance_custom_feat,height_custom_feat,reach_custom_feat,dob_diff_custom_feat,days_nofight_rol_sum_feat,days_nofight_exp_sum_feat,...,win_exp_max_min_d_feat,lose_exp_max_min_d_feat,wrest_w_exp_max_min_d_feat,wrest_l_exp_max_min_d_feat,KO_w_exp_max_min_d_feat,KO_l_exp_max_min_d_feat,dec_w_exp_max_min_d_feat,dec_l_exp_max_min_d_feat,fighter_dob_custom_feat,left_corner_custom_feat
0,Davey Grant,Da'Mon Blackshear,2025-07-26,Switch,Orthodox,-2.0,-3.0,-105.300000,609.0,3178.0,...,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,482.166667,0.0
1,Amanda Ribas,Tabatha Ricci,2025-07-26,Orthodox,Orthodox,2.0,5.0,-18.133333,175.0,707.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,388.566667,1.0
2,Nikita Krylov,Bogdan Guskov,2025-07-26,Orthodox,Orthodox,0.0,1.0,-6.300000,476.0,3654.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,406.466667,1.0
3,Shara Magomedov,Marc-Andre Barriault,2025-07-26,Orthodox,Orthodox,1.0,-1.0,51.600000,-35.0,-1631.0,...,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,0.0,379.800000,1.0
4,Bryce Mitchell,Said Nurmagomedov,2025-07-26,Orthodox,Southpaw,2.0,0.0,30.400000,-280.0,8.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.0,375.100000,0.0
5,Petr Yan,Marcus McGhee,2025-07-26,Orthodox,Switch,-1.0,-2.0,33.700000,154.0,1771.0,...,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,1.0,395.100000,1.0
6,Robert Whittaker,Reinier de Ridder,2025-07-26,Southpaw,Orthodox,-4.0,-5.0,3.466667,266.0,4348.0,...,1.0,1.0,-1.0,1.0,0.0,1.0,1.0,1.0,421.233333,1.0
